### Langchain tool calling & Agent

In [ ]:
!pip install langchain_google_genai langchain_core langchain_community tavily-python langgraph -q

In [9]:
import os
from dotenv import load_dotenv
from langchain_ollama import ChatOllama

load_dotenv('env', override=True)

model_name = 'qwen3:4b'

llm = ChatOllama(model = model_name, temperature=0.2, max_tokens=4096)

# llm = ChatOllama(model = model_name, temperature=0.2, max_tokens=4096, reasoning=True)
# reasoning 부분을 별도의 argument로 분리

llm.invoke("안녕")

AIMessage(content='<think>\nOkay, the user said "안녕" which means "Hello" in Korean. I need to respond appropriately.\n\nFirst, I should greet them back in Korean since they used Korean. So, "안녕하세요!" is a common greeting.\n\nWait, but maybe they want to switch to English? Hmm, but they started with Korean, so probably better to respond in Korean.\n\nLet me check the previous messages. The user just said "안녕" so there\'s no prior conversation. So I should respond with a friendly greeting.\n\nI should make sure the response is polite. In Korean, "안녕하세요" is the standard greeting. Alternatively, "안녕!" is also used but a bit more casual.\n\nSince the user might be in a context where they need a professional response, but given it\'s a simple hello, maybe "안녕하세요! 어떻게 도와드릴까요?" which means "Hello! How can I help you?" But maybe that\'s too much. Let\'s see.\n\nWait, the user just said "안녕" so the most basic response is "안녕하세요!" or "안녕!".\n\nBut in some contexts, adding a question to help them m

LangChain은 Tool Calling을 쉽게 연동하기 위한 기능을 제공합니다.   


LangChain에서 자체적으로 지원하는 Tool을 사용하거나, RAG의 Retriever를 Tool로 변환하는 것도 가능합니다.     
또한, 함수를 Tool로 변환할 수도 있습니다.

In [8]:
from langchain_tavily import TavilySearch
from langchain_core.tools import tool

@tool
def tavily_search(query, max_results=5):
    """Tavily API를 통해 검색 결과를 가져옵니다.
주어진 주제에 맞는 적절한 argument 값을 선정하세요.
query: 검색어
max_results : 검색 결과의 수(기본값은 5, 최대 20)"""
    tavily_search = TavilySearch(max_results=max_results)

    search_results = tavily_search.invoke(query)
    print(f"search_docs: {search_results}")

    context =''
    for doc in search_results['results']:
        doc_content = doc.get('content')

        context += 'TITLE: ' + doc.get('title','N/A') + '\nURL:' + doc.get('url')+ '\nContent:'+ doc_content
    return context

tavily_search.invoke("OpenAI GPT-OSS")

search_docs: {'query': 'OpenAI GPT-OSS', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://openai.com/es-ES/index/introducing-gpt-oss/', 'title': 'Presentamos gpt-oss', 'content': 'El modelo gpt-oss-20b ofrece resultados similares a OpenAI o3‑mini en pruebas comunes y puede ejecutarse en dispositivos edge con solo 16 GB de', 'score': 0.81665546, 'raw_content': None}, {'url': 'https://github.com/openai/gpt-oss', 'title': 'openai/gpt-oss', 'content': "Welcome to the gpt-oss series, OpenAI's open-weight models designed for powerful reasoning, agentic tasks, and versatile developer use cases. *   `chat` — a basic terminal chat application that uses the PyTorch or Triton implementations for inference along with the python and browser tools We include an inefficient reference PyTorch implementation in gpt_oss/torch/model.py. # Install the gpt-oss triton implementation python gpt_oss/metal/scripts/create-local-model.py -s <model_dir> -d <output_file> usag

'TITLE: Presentamos gpt-oss\nURL:https://openai.com/es-ES/index/introducing-gpt-oss/\nContent:El modelo gpt-oss-20b ofrece resultados similares a OpenAI o3‑mini en pruebas comunes y puede ejecutarse en dispositivos edge con solo 16 GB deTITLE: openai/gpt-oss\nURL:https://github.com/openai/gpt-oss\nContent:Welcome to the gpt-oss series, OpenAI\'s open-weight models designed for powerful reasoning, agentic tasks, and versatile developer use cases. *   `chat` — a basic terminal chat application that uses the PyTorch or Triton implementations for inference along with the python and browser tools We include an inefficient reference PyTorch implementation in gpt_oss/torch/model.py. # Install the gpt-oss triton implementation python gpt_oss/metal/scripts/create-local-model.py -s <model_dir> -d <output_file> usage: python -m gpt_oss.chat [-h] [-r REASONING_EFFORT] [-a] [-b] [--show-browser-results] [-p] [--developer-message DEVELOPER_MESSAGE] [-c CONTEXT] [--raw] [--backend {triton,torch,vllm}

이후, LLM은 Tool 사용이 필요한 경우 특별한 포맷의 메시지를 출력합니다.   
이를 Tool Call Message라고 부릅니다.

LLM은 프롬프트로 주어지는 툴 정보를 바탕으로 Tool의 사용을 결정합니다.    
Schema를 통해, 툴의 의미와 사용 방법, 형식을 이해합니다.

In [21]:
from langchain.prompts import ChatPromptTemplate

tools = [tavily_search]
llm_with_tools = llm.bind_tools(tools)

prompt = ChatPromptTemplate(
    [('system','주어진 툴을 사용해 사용자의 질문에 대답하세요.'),
    ('human','{query}')])

tool_chain = prompt | llm_with_tools
tool_call_msg = tool_chain.invoke("2025년 7월 22일 삼성전자 주가를 알려줘")
tool_call_msg

AIMessage(content='<think>\nOkay, the user is asking for the Samsung Electronics stock price on July 22, 2025. Let me think about how to approach this.\n\nFirst, I need to check if the Tavily search function can help here. The function allows searching the web with a query and max_results. But wait, the date mentioned is 2025, which is in the future. Current date is 2023, so July 22, 2025 hasn\'t happened yet. That\'s a problem.\n\nStock prices for future dates aren\'t available because they haven\'t occurred. The Tavily search might not have data for future dates either. So, the user is asking for something that doesn\'t exist yet. \n\nI should inform them that the date is in the future and stock prices can\'t be retrieved for that date. But the user might not realize that. Let me check the function parameters again. The function takes a query and max_results. Maybe if I search for "Samsung Electronics stock price July 22, 2025", but since it\'s a future date, the search results might

tool 에 tool_call 내용을 넣어 invoke 하면 ToolMessage 객체를 반환합니다. 해당 객체를 llm 에게 전달해야합니다.

In [23]:
from langchain_core.messages import HumanMessage

tool_list ={'tavily_search': tavily_search}

tool_name = tool_call_msg.tool_calls[0]['name']
tool_exec = tool_list[tool_name]

tool_response = tool_exec.invoke(tool_call_msg.tool_calls[0])    # ToolMessage
print(f"tool_response: {tool_response}")

messages = [
    HumanMessage(content="2025년 7월 22일 삼성전자 주가를 알려줘"),
    tool_call_msg,
    tool_response,
]

result = llm_with_tools.invoke(messages)
print(f"content: {result.content}")

search_docs: {'query': 'Samsung Electronics stock price July 22, 2025', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.investing.com/equities/samsung-electronics-co-ltd-historical-data', 'title': 'Samsung Electronics Co Stock Price History', 'content': 'Samsung Electronics Co Stock Price History ; Jul 22, 2025, 66,400, 66,200 ; Jul 21, 2025, 66,000, 68,100 ; Jul 20, 2025, 67,800, 67,400 ; Jul 17, 2025, 67,100', 'score': 0.95857275, 'raw_content': None}, {'url': 'https://finance.yahoo.com/quote/005930.KS/history/', 'title': 'Samsung Electronics Co., Ltd. (005930.KS) Stock Historical ...', 'content': 'KS) Stock Historical Prices & Data - Yahoo Finance. Oops, something went ... Jul 22, 2025, 68,100.00, 68,500.00, 65,600.00, 66,000.00, 66,000.00, 20,829,006.', 'score': 0.92715925, 'raw_content': None}, {'url': 'https://www.wsj.com/market-data/quotes/KR/XKRX/005930/historical-prices?gaa_at=eafs&gaa_n=ASWzDAgTdYeZuXB91nhtIgIheTQ4YwFCK6Gza6Mb7aAiSB

### [정리] Langchain tool calling 정리

In [27]:
from langchain_core.tools import tool

@tool
def multiply(x:int, y:int) -> int:
    "x와 y를 입력받아, x와 y를 곱한 결과를 반환합니다."
    return x*y

@tool
def current_date() -> str:
    "현재 날짜를 %y-%m-%d 형식으로 반환합니다."
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d")

def chat(llm, question , tools = [tavily_search]):
    tool_list = { x.name: x for x in tools }
    
    llm_with_tools = llm.bind_tools(tools)

    messages = [HumanMessage(content=question)]
    print('Query:', question)

    # LLM에 메시지 전달 (분기)
    tool_msg = llm_with_tools.invoke(question)
    messages.append(tool_msg)

    if tool_msg.content:
        print('LLM:', tool_msg.content)

    while tool_msg.tool_calls:
        for tool_call in tool_msg.tool_calls:
            tool_name = tool_call['name']

            print(f"-- {tool_name} 사용 중 --")
            print(tool_call)


            tool_exec = tool_list[tool_name]

            tool_result = tool_exec.invoke(tool_call)
            messages.append(tool_result)
        tool_msg = llm_with_tools.invoke(messages)
        messages.append(tool_msg)

        if tool_msg.content:
            print('LLM:', tool_msg.content)

    result = tool_msg

    return result.content

chat(llm, "OpenAI GPT-OSS 20B 모델을 쓰려면 GPU가 얼마나 필요해? /no-think", tools = [tavily_search, multiply, current_date])

Query: OpenAI GPT-OSS 20B 모델을 쓰려면 GPU가 얼마나 필요해? /no-think
LLM: <think>
Okay, let's see. The user is asking about the GPU requirements for using the OpenAI GPT-OSS 2:20B model. Hmm, first I need to check if I have the right information. Wait, OpenAI's main models are like GPT-3, GPT-3.5, but they don't have a GPT-OSS 20B model. Maybe the user is confused or referring to a different model. OSS might mean Open Source, so maybe they're talking about an open-source model that's 20B parameters.

Wait, the user wrote "OpenAI GPT-OSS 20B". OpenAI doesn't have an OSS model; their models are closed-source. Maybe it's a typo or misunderstanding. But assuming they mean an open-source 20B parameter model, like maybe Llama 2 or another model. But the user specifically says OpenAI, which is confusing.

Alternatively, maybe they're referring to a different model name. Let me think. The key here is that the user is asking about GPU requirements for a 20B parameter model. But since OpenAI doesn't have a

'<think>\nOkay, let\'s see. The user asked about the GPU requirements for OpenAI GPT-OSS 20B model. First, I need to check the search results provided.\n\nLooking at the first result from Reddit: It says that for the 20B model, a GPU is required, especially for better inference speeds (~80 tokens/s). But the title mentions "no is GPU required", which might be a typo. Wait, the content says "no is GPU required, especially for the 20B model, but having one significantly boosts inference speeds (~80 tokens/s)". So maybe it\'s saying that a GPU is needed, but without it, it\'s possible? Hmm, maybe the translation is a bit off.\n\nThe second result from Simon Willison\'s blog mentions that the gpt-oss-20b model can run on edge devices with just 16 GB of memory. So that\'s a key point. It says "can run on edge devices with just 16 GB of memory", which implies that the GPU needs to have at least 16 GB of VRAM? Or maybe the memory here refers to system memory?\n\nThird result from Medium: The 